In [1]:
#import packages
from mypackage.constants import SORTER, KK_DICT, FLAVOUR_DICT, SHOP_NAME
from mypackage.helpers import Helper
import pandas as pd
from datetime import date, timedelta
import os
import dataframe_image as dfi
import logging

import win32com.client
from PIL import ImageGrab

#Setup
pd.set_option('display.max_rows', 1000)

## KimSeng

In [4]:
def process_input_ks(df, pick_up_point_name,):
    '''
    clean df
    '''
    #https://stackoverflow.com/questions/23482668/sorting-by-a-custom-list-in-pandas/27255567
    # Create the dictionary that defines the order for sorting
    sorter_index = dict(zip(SORTER, range(len(SORTER))))

    # Generate a rank column that will be used to sort the dataframe numerically
    df['Point_Rank'] = df[pick_up_point_name].map(sorter_index, SORTER)
    
    # Here is the result asked with the lexicographic sort
    # Result may be hard to analyze, so a second sorting is proposed next
    df.sort_values('Point_Rank', ascending = True, inplace = True)
    df.drop('Point_Rank', 1, inplace = True)
    #insert a column of packet numbering
    df.insert(3, "Packet No.", range(1,len(df['Email Address'])+1))  
    #Generate a column for location name conversion
    df['Location'] = df[pick_up_point_name].map(KK_DICT, SORTER)
    return df

def output_text_ks(df, pick_up_point_name, output_text_path):
    # order list
    dishes = df[df.columns[6]].tolist()
    customizes = df[df.columns[7]].fillna(' ').tolist()
    with open(os.path.join(output_text_path, 'Order List.txt'), 'w', encoding="utf-8") as order_list:
        for index, (d, c) in enumerate(zip(dishes, customizes)):
            order_list.write(f'{index+1}. \n')
            for dish in d.split(','):
                dish = dish.strip()
                order_list.write(dish)
                order_list.write('\n')

            for customize in c.split(','):
                customize = customize.strip()
                order_list.write(customize + '\n')

            order_list.write('\n')

    unique_points = df[pick_up_point_name].unique()
    #for loop min() and max() of index of each location into txt file.    
    with open(os.path.join(output_text_path, 'Order Location.txt'), 'w', encoding="utf-8") as order_location:
        for point in unique_points:
            if df[df[pick_up_point_name]== point]['Packet No.'].min() == df[df[pick_up_point_name]== point]['Packet No.'].max():
                order_location.write(f"{df[df[pick_up_point_name]== point]['Packet No.'].min()}={KK_DICT[point]}"+"\n")
            else:
                order_location.write(f"{df[df[pick_up_point_name]== point]['Packet No.'].min()}-{df[df[pick_up_point_name]== point]['Packet No.'].max()}={KK_DICT[point]}"+"\n")


def generate_pickup_image_ks(df, excel_output_path, image_output_path):
    '''
    Style df > Export it > Generate image from the excel
    '''
    logging.info(f"Generating pickup image...")
    
    df_style = df.style.apply(Helper.highlight_rows_ks, axis=1)

    #Export as excel
    writer = pd.ExcelWriter(excel_output_path)
    df_style.to_excel(writer, sheet_name='Sheet1', index = False)

    #format for the column width, .set_column(index_start, index_end, width)
    # writer.sheets['Sheet1'].set_column(3, 3, 5)
    # writer.sheets['Sheet1'].set_column(4, 4, 30)
    # writer.sheets['Sheet1'].set_column(5, 5, 40)
    # writer.sheets['Sheet1'].set_column(6, 6, 45)

    writer.close()

    o = win32com.client.Dispatch('Excel.Application')
    wb = o.Workbooks.Open(os.path.join(os.getcwd(), excel_output_path))
    ws = wb.Worksheets['Sheet1']

    ws.Range(ws.Cells(1,1),ws.Cells(df.shape[0],df.shape[1])).CopyPicture(Format=2)

    img = ImageGrab.grabclipboard()
    img.save(image_output_path)
    wb.Close(True)

    logging.info(f"Image generated in {image_output_path}!")

def run_ks(date_str):
    shop_name = SHOP_NAME['KS'] #or 'JS'
    #create folders
    Helper.create_folders(shop_name, date_str)
    pick_up_point_name = "KimSeng Food Pick Up Point"
    shop_input_path = os.path.join(shop_name, 'input')
    shop_output_path = os.path.join(shop_name, 'output', date_str)

    excel_output_path = os.path.join(shop_output_path, f"{date_str} orders.xlsx")
    image_output_path = os.path.join(shop_output_path, f"{date_str} image.jpg")
    
    for file in os.listdir(shop_input_path):
        #if today file exist:
        if date_str in file:
            logging.info(f"Today's {file} found! cleaning the df...")
            df = pd.read_csv(os.path.join(shop_input_path, file))
            df = process_input_ks(df, pick_up_point_name)
            # display(df)
            output_text_ks(df, pick_up_point_name, shop_output_path) ##print text
            generate_pickup_image_ks(df, excel_output_path, image_output_path)
            break
        
        logging.info(f"{date_str} file not found in {shop_name}. Exiting the program...")

In [5]:
# today_str = date.today().strftime('%d_%m_%Y')
today_str = date.today().strftime('27_10_2022')
run_ks(today_str)

C:\Users\USER\AppData\Local\Temp\ipykernel_17472\3517983203.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop('Point_Rank', 1, inplace = True)
